In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import socket
import tp_machine as TPm


Current_DIR = os.getcwd()
TPBench_DIR = os.path.dirname(os.path.dirname(Current_DIR))

def draw_figure(df=pd.DataFrame(), serial_title='', x_label='', y_label='', x_series=[], log_base=2):
    plt.figure()
    for index, row in df.iterrows():
        plt.plot(x_series, row[1:], marker='o', label=row[0])
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    if log_base > 1:
        plt.xscale('log', base=log_base)
    plt.title(serial_title)
    plt.legend()
    plt.grid(True)
    plt.show()


def draw_blas(csv_dir, kernels, headers=None):
    blas_dir = os.path.join(csv_dir, 'blas1')
    for csv_file in os.listdir(blas_dir):
        if csv_file.endswith('.csv'):
            csv_name = os.path.splitext(csv_file)[0]
            csv_path = os.path.join(blas_dir, csv_file)
            df = pd.read_csv(csv_path, header=None)
            # df.coloumns = headers
            serial_title = f'{csv_name}'
            x_label = 'size (KB)'
            y_label = 'median B/c'
            x_series = [int(x) for x in headers[1:]]
            df = df[df[0].isin(kernels)]
            # print(df)

            draw_figure(df, serial_title, x_label, y_label, x_series, log_base=2)

def draw_roofline(csv_dir, headers=None):
    roofline_dir = os.path.join(csv_dir, 'roofline')
    for csv_file in os.listdir(roofline_dir):
        if csv_file.endswith('.csv'):
            csv_name = os.path.splitext(csv_file)[0]
            csv_path = os.path.join(roofline_dir, csv_file)
            if os.stat(csv_path).st_size == 0:
                continue
            df = pd.read_csv(csv_path, header=None)
            # df.coloumns = headers
            serial_title = f'{csv_name}'
            x_label = 'Compute/LD Ratio'
            y_label = 'flops/c'
            x_series = [float(x) for x in headers[1:]]

            np1_series = df[df[0].str.startswith('np1-')]
            npM_series = df[~df[0].str.startswith('np1-')]
            
            first_value = npM_series.iloc[0, 0]
            npM = int(re.search(r'np(\d+)', first_value).group(1)) if re.search(r'np(\d+)', first_value) else None
            
            draw_figure(np1_series, serial_title + ' 1 process', x_label, y_label, x_series, log_base=2)
            draw_figure(npM_series, serial_title + f' {npM} process', x_label, y_label, x_series, log_base=2)

def draw_stencil(csv_dir, headers=None):
    stencil_dir = os.path.join(csv_dir, 'stencil')
    for csv_file in os.listdir(stencil_dir):
        if csv_file.endswith('.csv'):
            csv_name = os.path.splitext(csv_file)[0]
            csv_path = os.path.join(stencil_dir, csv_file)
            df = pd.read_csv(csv_path, header=None)
            # df.coloumns = headers
            serial_title = f'{csv_name}'
            x_label = 'Matrix Row Length with 100 Col Length (# of Double)'
            y_label = 'median B/c'
            x_series = [int(x) for x in headers[1:]]

            np1_series = df[df[0].str.startswith('np1-')]
            npM_series = df[~df[0].str.startswith('np1-')]
            first_value = npM_series.iloc[0, 0]
            npM = int(re.search(r'np(\d+)', first_value).group(1)) if re.search(r'np(\d+)', first_value) else None

            draw_figure(np1_series, serial_title+ f' {1} process', x_label, y_label, x_series, log_base=10)
            draw_figure(npM_series, serial_title + f' {npM} process', x_label, y_label, x_series, log_base=10)



if __name__ == '__main__':
    result_dir = f'result/{TPm.machine}/summary'
    csv_dir = os.path.join(TPBench_DIR, result_dir)

    blas_kernels=['d_init', 'd_sum', 'd_copy', 'd_update', 'd_triad', 'd_striad', 'd_axpy', 'd_scale', 'd_staxpy']
    # blas_kernels=['staxpy']
    headers = ['title', '4', '8', '16', '32', '64', '128', '256', '384', '512', '768', '1024', '1536', '2048', '3072', '4096', '8169', '16484','32768', '65536', '102400', '204800', '409600']
    draw_blas(csv_dir, blas_kernels, headers)

    headers = ['title', '0.25', '0.5', '1', '2', '4', '8']
    draw_roofline(csv_dir, headers)

    headers = ['title','100', '1024', '10240', '102400', '1024000']
    draw_stencil(csv_dir, headers)
